In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
#!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab
%cd /content/
# install detectron2:
!git clone https://github.com/facebookresearch/detectron2
%cd /content/detectron2
!pip install -r requirements.txt
!python setup.py install
!pip install git+https://github.com/facebookresearch/fvcore.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-3yrxhopb
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-3yrxhopb
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done
2.0.0+cu118 True
gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

/content
fatal: destination path 'detectron2' already exists and is not an empty directory.
/content/detectron2
ERROR: Could not open requirements file: [Errno 2] No such file or direct

In [1]:
import torch
from torchvision.transforms.functional import to_tensor
from torchvision.transforms import Resize
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torchvision.transforms.functional as TF
import torch.nn as nn
import torchvision
import torch
import detectron2
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog
from detectron2.data import DatasetCatalog
from detectron2.data import transforms as T
from detectron2.structures import BoxMode
from detectron2.modeling import build_model
from PIL import Image, ImageDraw

# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# # import some common libraries
# import numpy as np
# import cv2
# import random
# from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

d:\Clase\TFG\env\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'D:\Clase\TFG\env\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [49]:
class PoolDetectionModel(nn.Module):
    def __init__(self, num_classes):
        super(PoolDetectionModel, self).__init__()

        num_predictions = 10
        
        # Cargamos el modelo pre-entrenado
        self.model = torch.hub.load('facebookresearch/detr:main', 'detr_resnet50', pretrained=True)

        # Agregamos una nueva capa class_embed
        hidden_dim = self.model.transformer.d_model
        self.model.class_embed = nn.Linear(hidden_dim, num_classes)
        
        # Cambiamos el número de clases de salida
        num_query = self.model.transformer.d_model
        self.model.num_classes = num_classes
        self.model.query_embed = nn.Embedding(num_predictions, num_query)
        
    def forward(self, x):
        # Pasamos las imágenes por el modelo pre-entrenado
        outputs = self.model(x)
        
        # Obtenemos las cajas y las etiquetas de las predicciones
        boxes = outputs['pred_boxes']
        labels = outputs['pred_logits'].softmax(-1)
        
        # Devolvemos las cajas y las etiquetas
        return boxes, labels

In [50]:
import torch
from scipy.optimize import linear_sum_assignment
import torch.nn.functional as F

def hungarian_loss(outputs, targets):
    print(outputs)
    print(targets)
    boxes, labels = outputs
    true_boxes, true_labels = torch.FloatTensor(targets['boxes']).to(device), torch.FloatTensor(targets['labels']).to(device)
    
    # Calculamos el costo de emparejamiento entre las cajas predichas y verdaderas
    cost_boxes = torch.cdist(boxes, true_boxes, p=1)
    
    # Calculamos el costo de emparejamiento entre las etiquetas predichas y verdaderas
    cost_labels = torch.cdist(labels, true_labels, p=1)
    
    # Combinamos los costos
    cost = cost_boxes + cost_labels
    
    # Resolvemos el problema de asignación lineal
    row_ind, col_ind = linear_sum_assignment(cost.cpu().detach().numpy())
    
    # Obtenemos las cajas y etiquetas predichas emparejadas
    boxes = boxes[row_ind]
    labels = labels[row_ind]
    
    # Obtenemos las cajas y etiquetas verdaderas emparejadas
    true_boxes = true_boxes[col_ind]
    true_labels = true_labels[col_ind]
    
    # Calculamos la pérdida de IoU para las cajas
    box_loss = F.mse_loss(boxes, true_boxes)
    
    # Calculamos la entropía cruzada binaria para las etiquetas
    label_loss = F.binary_cross_entropy(labels, true_labels)
    
    # Combinamos las pérdidas
    loss = box_loss + label_loss
    
    return loss

In [51]:
import torch
import torch.nn.functional as F

def object_detection_loss(output, target):
    bbox_pred = torch.squeeze(output[0])
    prob_pred = torch.squeeze(output[1])

    bbox_true = torch.FloatTensor([t["boxes"][0] for t in target]).to(device)

    # prob_true = torch.FloatTensor([label for t in target for label in t["labels"]]).to(device)
    prob_true = torch.FloatTensor([t["labels"][0] for t in target]).to(device)


    # Calcula la pérdida de entropía cruzada con pesos para la clasificación
    classification_loss = F.binary_cross_entropy(prob_pred, prob_true)

    # Recorte provisional de las bbtrue
    

    # Calcula la pérdida de error cuadrático medio para la regresión
    regression_loss = F.mse_loss(bbox_pred, bbox_true)

    # Combina las pérdidas de clasificación y regresión
    loss = classification_loss + regression_loss

    return loss

In [56]:
def train(model, optimizer, data_loader, device):
    model.train()
    for name, param in model.named_parameters():
        if "query_embed" not in name and "bbox_embed" not in name:
            param.requires_grad = False
    for epoch in range(10):
      for images, targets in data_loader:
          images = [TF.to_tensor(image).to(device) for image in images]
          targets = [{k: v for k, v in t.items()} for t in targets]
          output = model(images)
          print(output[0].shape)
          print(output[0][0])
          print(output[1].shape)
          print(output[1][0])
          # losses = 0
          # output = [(prob, targ) for prob, targ in zip(output[0], output[1])]
          # for o, t in zip(output, targets):
          #   losses += hungarian_loss(o, t)
          losses = object_detection_loss(output, targets)
          print(losses)
          optimizer.zero_grad()
          losses.backward()
          optimizer.step()
          
    return losses.item()

In [57]:
def collate_fn(batch):
    images = []
    targets = []
    new_size = (100, 100)
    transform = Resize(new_size)
    for sample in batch:
        image = Image.open(sample['file_name']).convert('RGB')
        width, height = image.size
        # Transformar la imagen
        image = transform(image)
        # Adaptar las anotaciones
        annotations = sample['annotations']
        for ann in annotations:
            bbox = ann['bbox']
            x_original, y_original, w_original, h_original = bbox
            x_new = x_original * new_size[0] / width
            y_new = y_original * new_size[1] / height
            w_new = w_original * new_size[0] / width
            h_new = h_original * new_size[1] / height
            ann['bbox'] = [x_new, y_new, w_new, h_new]
        
        # Añadir la imagen y las anotaciones a la lista
        images.append(image)
        targets.append({'boxes': [ann['bbox'] for ann in annotations], 'labels': [1 for ann in annotations if ann['category_id'] == 0]})
    return images, targets

In [58]:
DatasetCatalog.clear()
MetadataCatalog.clear()

# Declarar el modelo
model = PoolDetectionModel(num_classes=1)

# Declarar el optimizador
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Registrar instancia de COCO
register_coco_instances("piscinas_entrenamiento", {}, "..\\data\Zona\Dataset\\annotations.json", "..\\data\Zona\Dataset\images")

# Cargar la instancia de COCO como un objeto Dataset
dataset_name = "piscinas_entrenamiento"
dataset = DatasetCatalog.get(dataset_name)

# Obtener metadatos del dataset
metadata = MetadataCatalog.get(dataset_name)

# Declarar el DataLoader
data_loader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)


# Declarar el dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Mover el modelo y el DataLoader al dispositivo
model.to(device)
train(model, optimizer, data_loader, device)

Using cache found in C:\Users\rober/.cache\torch\hub\facebookresearch_detr_main


WARNING [04/12 02:09:52 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[04/12 02:09:52 d2.data.datasets.coco]: Loaded 7355 images in COCO format from ..\data\Zona\Dataset\annotations.json
torch.Size([32, 10, 4])
tensor([[0.6082, 0.9063, 0.6799, 0.1952],
        [0.9298, 0.5642, 0.1385, 0.8483],
        [0.1494, 0.9601, 0.1428, 0.0831],
        [0.8559, 0.5486, 0.2913, 0.8957],
        [0.9780, 0.2758, 0.0441, 0.3558],
        [0.9678, 0.3642, 0.0634, 0.1914],
        [0.9827, 0.3096, 0.0346, 0.2876],
        [0.9809, 0.2786, 0.0384, 0.2705],
        [0.9722, 0.3023, 0.0554, 0.3049],
        [0.9542, 0.5554, 0.0918, 0.8900]], grad_fn=<SelectBackward0>)
torch.Size([32, 10, 1])
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], grad_fn=<SelectBackward0>)


ValueError: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 10])) is deprecated. Please ensure they have the same size.

In [ ]:
torch.save(model, "/content/drive/MyDrive/TFG/model.pt")